Recreating SIR+Q Model Presented by Rizi A, Keating L, Gleeson J et al in their study 'The Unreasonable Effectiveness of Contact Tracing on Networks with' Cliques' in Epydemic

Link to paper: https://arxiv.org/abs/2304.10405v2

Link to repo: https://github.com/k-rizi/Contact-Tracing-On-Clique-Networks


In [ ]:
from epyc import JSONLabNotebook, ParallelLab
import epyc
import networkx as nx
import random, itertools
from random import shuffle
import epydemic
import numpy as np
from epydemic import SIR, Monitor, ProcessSequence, rng, ERNetwork, SEIR, SIR_FixedRecovery, FixedNetwork
import matplotlib.pyplot as plt
from parameters import *

In [ ]:
create_data_output_directory()

### Quarantine First then Infect SIR+Q Model

In [ ]:
class SIRQ(SIR):
    P_DETECT = 'pDetect'

    QUARANTINE = 'epydemic.sir.Q'
    
    INFECTION_TIME = 'infectionTime'  

    def __init__(self):
        super().__init__()

    def build(self, params):
        pI = params[self.P_INFECT]
        pD = params[self.P_DETECT]
        pR = params[self.P_REMOVE]
        
        # Set Population so all node are initially within the S compartment. The function initialCompartments will then infect a single random node before the simulation begins. 
        self.addCompartment(self.SUSCEPTIBLE, 1)
        self.addCompartment(self.INFECTED, 0)
        self.addCompartment(self.REMOVED, 0.0)
        self.addCompartment(self.QUARANTINE, 0.0)

        self.trackNodesInCompartment(self.INFECTED)
        self.trackEdgesBetweenCompartments(self.SUSCEPTIBLE, self.INFECTED, name=self.SI)

        # Attach Quarantine event to SI loci
        self.addEventPerElement(self.SI, pD, self.quarantine, name=self.QUARANTINE)
        
        # The Infection Event Follows the Quarantine Event
        self.addEventPerElement(self.SI, pI, self.infect, name=self.INFECTED)
        
        # Finally the Infected Node is Removed
        self.addEventPerElement(self.INFECTED, pR, self.remove, name=self.REMOVED)
        

    def atEquilibrium(self, t):
        return len(self.compartment(self.INFECTED)) == 0

    def quarantine(self, t ,e):
        (n, _ ) = e
        self.changeCompartment(n, self.QUARANTINE)

    def initialCompartments(self):
        g = self.network()
        ns = set(g.nodes())
        N = len(ns)

        n = rng.integers(N)
        self.changeCompartment(n, self.INFECTED)
        g.nodes[n][self.INFECTION_TIME] = 0.0

        # mark all other nodes as susceptible
        ns.remove(n)
        for n in ns:
            self.changeInitialCompartment(n, self.SUSCEPTIBLE)

### Quarantine Infection First then Quarantine SIR+Q Model

In [ ]:
class SIRQ_Infection_First(SIR):
    P_DETECT = 'pDetect'

    QUARANTINE = 'epydemic.sir.Q'
    
    INFECTION_TIME = 'infectionTime'  

    def __init__(self):
        super().__init__()

    def build(self, params):
        pI = params[self.P_INFECT]
        pD = params[self.P_DETECT]
        pR = params[self.P_REMOVE]

        self.addCompartment(self.SUSCEPTIBLE, 1)
        self.addCompartment(self.INFECTED, 0)
        self.addCompartment(self.REMOVED, 0.0)
        self.addCompartment(self.QUARANTINE, 0.0)

        self.trackNodesInCompartment(self.INFECTED)
        self.trackEdgesBetweenCompartments(self.SUSCEPTIBLE, self.INFECTED, name=self.SI)

        # Trigger the infection event first
        self.addEventPerElement(self.SI, pI, self.infect, name=self.INFECTED)
        
        # After infection, fire the quarantine event.
        self.addEventPerElement(self.SI, pD, self.quarantine, name=self.QUARANTINE)
        
        self.addEventPerElement(self.INFECTED, pR, self.remove, name=self.REMOVED)
        

    def atEquilibrium(self, t):
        return len(self.compartment(self.INFECTED)) == 0

    def quarantine(self, t ,e):
        (n, _ ) = e
        self.changeCompartment(n, self.QUARANTINE)

    def initialCompartments(self):
        g = self.network()
        ns = set(g.nodes())
        N = len(ns)

        n = rng.integers(N)
        self.changeCompartment(n, self.INFECTED)
        g.nodes[n][self.INFECTION_TIME] = 0.0

        # mark all other nodes as susceptible
        ns.remove(n)
        for n in ns:
            self.changeInitialCompartment(n, self.SUSCEPTIBLE)

### Create a subclass of the SynchronousDynamics class to randomise the order of events in each timestep. 

In [ ]:
class SynchronousDynamics_RandomOrder(epydemic.SynchronousDynamics):
    def __init__(self, p, g):
        super().__init__(p, g)

    def allEventsInTimestep(self, t):
        res = super().allEventsInTimestep(t)
        shuffle(res)
        return res

### Build epyc experiements to define the parameter space and run simulations

In [ ]:
def ex_1_sirq(lab):
    lab[SIRQ.P_INFECT] = p_infect
    lab[SIRQ.P_DETECT] = p_detects
    lab[SIRQ.P_REMOVE] = p_remove

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    lab['ens'] = range(ens)

    # create the model, network generator, and experiment
    p = SIRQ()
    g = ERNetwork()
    e = epydemic.SynchronousDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

In [ ]:
def ex_2_sirq_IQ(lab):
    lab[SIRQ_Infection_First.P_INFECT] = p_infect
    lab[SIRQ_Infection_First.P_DETECT] = p_detects
    lab[SIRQ_Infection_First.P_REMOVE] = p_remove

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    lab['ens'] = range(ens)

    # create the model, network generator, and experiment
    p = SIRQ_Infection_First()
    g = ERNetwork()
    e = epydemic.SynchronousDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

In [ ]:
def ex_3_sirq_STO(lab):
    lab[SIRQ.P_INFECT] = p_infect
    lab[SIRQ.P_DETECT] = p_detects
    lab[SIRQ.P_REMOVE] = p_remove

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    lab['ens'] = range(ens)

    # create the model, network generator, and experiment
    p = SIRQ()
    g = ERNetwork()
    e = epydemic.StochasticDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

In [ ]:
def ex_4_sirq_RDM(lab):
    lab[SIRQ.P_INFECT] = p_infect
    lab[SIRQ.P_DETECT] = p_detects
    lab[SIRQ.P_REMOVE] = p_remove

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    lab['ens'] = range(ens)

    # create the model, network generator, and experiment
    p = SIRQ()
    g = ERNetwork()
    e = SynchronousDynamics_RandomOrder(p, g)

    # run the experiment
    lab.runExperiment(e)

In [ ]:
lab = ParallelLab(JSONLabNotebook(get_out_path('ex_1_sirq_QI'), create=True), cores)
lab.createWith("ex_1_sirq_QI", ex_1_sirq)

In [ ]:
lab = ParallelLab(JSONLabNotebook(get_out_path('ex_2_sirq_IQ'), create=True), cores)
lab.createWith("ex_2_sirq_IQ", ex_2_sirq_IQ)

In [ ]:
lab = ParallelLab(JSONLabNotebook(get_out_path('ex_3_sirq_STO'), create=True), cores)
lab.createWith("ex_3_sirq_STO", ex_3_sirq_STO)

In [ ]:
lab = ParallelLab(JSONLabNotebook(get_out_path('ex_4_sirq_RDM'), create=True), cores)
lab.createWith("ex_4_sirq_RDM", ex_4_sirq_RDM)